In [2]:
import pandas as pd
import re
from config.fucntions import img_downloader as imgd
from config.fucntions import columns_converter as cc
import os

# Preprocessing

In [3]:
file_name = '../datasets/production/dataset_CLEAN.csv'
df = pd.read_csv(file_name, sep=';', encoding='utf-8')
file_name = '../config/variables/blueprint_prefixes.txt'
with open(file_name, 'r') as f:
    blueprint_prefixes  = f.read().splitlines()

file_name = '../datasets/production/dataset_MVP.csv'
df_mvp = pd.read_csv(file_name, sep=';', encoding='utf-8')

# # Takes ~20 seconds
cc.strings_to_links(df_mvp, df_mvp.columns[1:3])
cc.strings_to_links(df, df.columns[3:])

## Downloading. Option 1

Target column (Prof.Bell) is `image_links`. However, in column `microfiche_links` there are **scans** of photos and documents  

In order to download pictures to your computer, run all code in **Preprocessing** section and then go to section **Downloading** and choose whether you want to download images from `images_links` or scnas from `microfiche_links` by setting relevant variables. Don't forget to specify amount of pictures to download `OBJECTS_TO_DOWNLOAD`

Converting strings in coulmn image_links - microfiche_archive_links back into lists of strings

In [4]:
df.head()

,id_record,id_persistent,label,image_links,archive_links,microfiche_links,microfiche_archive_links
0,http://www.bildindex.de/document/obj20666124,http://id.bildindex.de/thing/0001618283,"Lauenhain (Kreis Hainichen), Fahnenträger eine...",[],[],[http://www.bildindex.de/bilder/d/mi12308g07],[https://www.bildindex.de/media/obj20666124/mi...
1,http://www.bildindex.de/document/obj20727600,http://id.bildindex.de/thing/0001677732,Fenster (Bauelement),[http://www.bildindex.de/bilder/d/fm140030],[https://www.bildindex.de/media/obj20727600/fm...,"[http://www.bildindex.de/bilder/d/mi02117a05, ...",[https://www.bildindex.de/media/obj20727600/mi...
2,http://www.bildindex.de/document/obj20943303,http://id.bildindex.de/thing/0001803584,"Kassel, Infanteriekasernen an der Königsstraße...",[http://www.bildindex.de/bilder/d/STMP_II_1108...,[https://www.bildindex.de/media/obj20943303/ST...,[],[]
3,http://www.bildindex.de/document/obj20948462,http://id.bildindex.de/thing/0001806337,"Waldeck, Domäne, Wohnhaus, Umbauentwurf, Aufrisse",[http://www.bildindex.de/bilder/d/STMP_II_1566...,[https://www.bildindex.de/media/obj20948462/ST...,[],[]
4,http://www.bildindex.de/document/obj20943569,http://id.bildindex.de/thing/0001803733,"Kassel, Opernhaus (Altes Hoftheater), Aborte i...",[http://www.bildindex.de/bilder/d/STMP_II_1383...,[https://www.bildindex.de/media/obj20943569/ST...,[],[]


In [5]:
# print(f'Amount of objects in dataset: {df.shape[0]}')
# print(f'Images to download: {df.image_links.apply(lambda x: len(x)).sum()}')
# print(f'Scans to download: {df.microfiche_links.apply(lambda x: len(x)).sum()}')
# print()
# print('(with blueprints)')


In [6]:
# N_img_clean, N_obj_clean = 0, 0
# blueprint_suffix = ['_r', '_v', 'T1', 'T2', 'V2', 'V1']

# for i in range(df.shape[0]):
    
#     num_img = len(df.iloc[i].image_links)

#     if num_img == 0:    
#         continue
    
#     if df.iloc[i].image_links[0][-2:] not in blueprint_suffix:
#         N_img_clean += num_img
#         N_obj_clean += 1


# print(f'Amount of objects in dataset with images available: {N_obj_clean}')
# print(f'Amount of images in \"image_links\": {N_img_clean}')
# print()
# print('(without blueprints)')


In [7]:
# blueprint_suffix = ['_r', '_v', 'T1', 'T2', 'V2', 'V1']

# b_not_s, b_s, not_b_s, imgs = 0, 0, 0, 0

# for i in range(df.shape[0]):  
#     k_blueprints, k_scans, k_img = 0,  len(df.iloc[i]['microfiche_links']), 0

#     for link in df.iloc[i]['image_links']:
        
#         if any(prefix in link.split('/')[-1] for prefix in blueprint_prefixes):
#         # if link[-2:] in blueprint_suffix:
#             k_blueprints += 1
#         else:
#             k_img += 1
            
#     if k_img:
#         imgs += k_img
#     elif not k_blueprints:
#         imgs += k_scans
        
#     if k_blueprints and k_scans:
#         b_s += 1
#     elif k_blueprints and not k_scans:
#         b_not_s += 1
#     elif not k_blueprints and k_scans:
#         not_b_s += 1

# print(f'Blueprints and scans: {b_s}')
# print(f'Blueprints only: {b_not_s}')
# print(f'Scans only: {not_b_s}')
# print(f'Images (image or scan, when there is no image): {imgs}')

There are only 7 objects with both blueprints and scans presented. After manual checking we understood, that they are not related to our task, so we can skip them.

In [8]:
# # One object consists of many images. Max = df.shape[0]
# OBJECTS_TO_DOWNLOAD = 100

# # If you want to download only scans, set this to True
# is_scan_only = False

# # If you want to download only blueprints, set this to True
# is_blueprints_only = True

# # Otherwise it will download no blueprints, only images ('image_links') or scans, when images are absent

In [9]:
# if is_scan_only:
#     folder_name, column, file_type = 'scans_only', 'microfiche_links', 'scn'
#     is_blueprints_only, is_all = False, False
# elif is_blueprints_only:
#     folder_name, column, file_type = 'blueprints_only', 'image_links', 'blpnt'
#     is_all = False
# else:
#     folder_name, file_type = 'images', 'img'
#     is_all = True
    
# imgd.create_folder('images/' + folder_name)

# for i in range(OBJECTS_TO_DOWNLOAD):

#     # For objects without images check scans availability
#     if is_all:
#         if not len(df.iloc[i]['image_links']):
#             if not len(df.iloc[i]['microfiche_links']):
#                 continue
#             else:
#                 column, file_type = 'microfiche_links', 'scn'
#         else:
#             column, file_type = 'image_links', 'img'

#     # Iteration through lists of links
#     for link in df.iloc[i][column]:

#         # Skip blueprints file if needed
#         if is_blueprints_only:
#             if any(prefix not in link.split('/')[-1] for prefix in blueprint_prefixes):
#                 continue
#         else:
#             if any(prefix in link.split('/')[-1] for prefix in blueprint_prefixes):
#                 continue
            
#         # Build a name for the image from id_persistent and link
#         name = ''
#         line = df.iloc[i].id_persistent
#         if line != 'NO_ID_PERSISTENT':
#             name += re.findall(r'.*/([^/]*)$', line)[0] + '-'
#         else:
#             name += line+'-'
#         name += re.findall(r'.*/([^/]*)$', link)[0]

#         # Download the image
#         save_path = f'images/{folder_name}/{file_type}-{name}.jpg'
#         try:
#             response, status = imgd.download_image(link, save_path)
#             if response == 'Failed':
#                 if file_type == 'img':

#                     # Try to download same object from scans
#                     for link_new in df.iloc[i]['microfiche_links']:
#                         # Skip blueprints file
#                         if any(prefix in link.split('/')[-1] for prefix in blueprint_prefixes):
#                             continue
                        
#                         # Build a new name for the image
#                         name = name[:re.search(f'-', name).span()[0]+1]
#                         name += re.findall(r'.*/([^/]*)$', link_new)[0]
#                         save_path = f'images/{folder_name}/scn-{name}.jpg'

#                         # Download the image
#                         response, status = imgd.download_image(link_new, save_path)
#                         if response == 'Failed':
#                             print(f'Failed downloading both image ({link}) and scan ({link_new}) of object {i}. Status code: {status}')
#                         continue
#                 # print(f'Failed downloading image {link} of object {i}. Status code: {status}')

#             if response == 'Error':
#                 print(f'Error downloading image {link} of object {i}. Status code: {status}')
#         except:
#             print(f'Error downloading image {link} of object {i}')

In [10]:
# pic = 'img-0001350498-fm308202'
# i,column,j = imgd.filename_to_object(df, pic)
# df.iloc[i][column][j], i, j, column

In [11]:
# i=1000
# imgd.object_to_filenames(df, i), imgd.link_to_filename(df.iloc[i].id_persistent, df.iloc[i].image_links[0], 'img')

## Downloading. Option 2 (recomended)
For task, where pictures and scans are involved

In [12]:
print(f'Images available to download: {df_mvp.shape[0]}')
print(f'All unique: {df_mvp.id.unique().shape[0]==df_mvp.shape[0]}')
df_mvp.head()

Images available to download: 37185
All unique: True


,link,id_objects,labels,file_name,id
0,http://www.bildindex.de/bilder/d/ae00001b05,[9077],[61],scn-ae00001b05,ae00001b05
1,http://www.bildindex.de/bilder/d/ae00001b06,[9077],[61],scn-ae00001b06,ae00001b06
2,http://www.bildindex.de/bilder/d/ae00001b07,[9077],[61],scn-ae00001b07,ae00001b07
3,http://www.bildindex.de/bilder/d/ae00001b08,[9077],[61],scn-ae00001b08,ae00001b08
4,http://www.bildindex.de/bilder/d/ae00001b10,[3899],[25],scn-ae00001b10,ae00001b10


In [13]:
OBJECTS_TO_DOWNLOAD = df_mvp.shape[0]

In [ ]:
folder_name = 'images'
imgd.create_folder(f'images/{folder_name}')

file_names = os.listdir('images/images')
file_names = [f for f in file_names if os.path.isfile(os.path.join('images/images', f))]

for i in range(OBJECTS_TO_DOWNLOAD):
        
        # image_to_skip is defined further. Sorry for this inconvenience, 
        # I was lazy to put it into separate dataset
        if df_mvp.iloc[i].id in image_to_skip:
             continue

        file_name = df_mvp.iloc[i].file_name
        link = df_mvp.iloc[i].link
        save_path = f'images/{folder_name}/{file_name}.jpg'

        try:
            response, status = imgd.download_image(link, save_path)
            if response == 'Failed' or response == 'Error':
                print(f'Error downloading image {link} of object {df_mvp.iloc[i].id}. Status code: {status}')
        except:
            print(f'Error downloading image {link} of object {df_mvp.iloc[i].id}')

### Post downloading investigation

In [38]:
folder_path = 'images/images'
file_names = os.listdir(folder_path)
# file_names = [f for f in file_names if os.path.isfile(os.path.join(folder_path, f))]
num_scans = len([file for file in file_names if 'scn' in file])

print(f'Amount of files dowloaded: {len(file_names)} ({round(len(file_names)/df_mvp.shape[0]*100,2)}%)')
print(f'Scans: {num_scans} ({round(num_scans/len(file_names)*100,2)}%)')
print(f'Images: {len(file_names) - num_scans} ({round((len(file_names) - num_scans)/len(file_names)*100,2)}%)')
print()
a = round((df_mvp.shape[0]-len(file_names))/df_mvp.shape[0]*100,2)
print(f'Lost files due to absence of image under the link: {df_mvp.shape[0]-len(file_names)} ({a}%)')

# file_names[:5]
# file_names

Amount of files dowloaded: 36983 (99.46%)
Scans: 17455 (47.2%)
Images: 19528 (52.8%)

Lost files due to absence of image under the link: 202 (0.54%)


###  Investigating the 404 errors of some links

In [19]:
with open('../config/errors/downloading_failed_404.txt', 'r') as f:
    responses_404  = f.read().splitlines()

responses_404 = [response for response in responses_404 if 'Error downloading image' in response]
print(f'All responses are 404: {all(response[-3:] == "404" for response in responses_404)}. Those are:')
print([response for response in responses_404 if response[-3:] != "404"])

# Save all responses in list of tupels (link, object_id)
pattern = r'image (\S+) of object (\S+)'
responses_404 = [re.search(pattern, response).groups() for response in responses_404 if re.search(pattern, response)]
responses_404 = [(response[0], re.sub(r'\.', '', response[1])) for response in responses_404]

# Save all ids of images, which failed downloading
image_to_skip = []
for response in responses_404:
    image_to_skip.append(response[1])

All responses are 404: False. Those are:
['Error downloading image http://www.bildindex.de/bilder/d/cv4196 of object cv4196', 'Error downloading image http://www.bildindex.de/bilder/d/cv4295 of object cv4295', 'Error downloading image http://www.bildindex.de/bilder/d/fm810041 of object fm810041', 'Error downloading image http://www.bildindex.de/bilder/d/fm811033a of object fm811033a', 'Error downloading image http://www.bildindex.de/bilder/d/fm810385 of object fm810385', 'Error downloading image http://www.bildindex.de/bilder/d/fmbc31763_09 of object fmbc31763_09', 'Error downloading image http://www.bildindex.de/bilder/d/fmb22348_16 of object fmb22348_16', 'Error downloading image http://www.bildindex.de/bilder/d/fmc431641 of object fmc431641', 'Error downloading image http://www.bildindex.de/bilder/d/fmd10008163 of object fmd10008163', 'Error downloading image http://www.bildindex.de/bilder/d/fmd10012875 of object fmd10012875', 'Error downloading image http://www.bildindex.de/bilder/

In [15]:
len(image_to_skip)

202

In [97]:
df.iloc[df_mvp[df_mvp.id == 'cv13290'].id_objects.iloc[0]]

,id_record,id_persistent,label,image_links,archive_links,microfiche_links,microfiche_archive_links
3269,http://www.bildindex.de/document/obj06313290,http://id.bildindex.de/thing/0002088697,Hl. Johannes Baptista,[http://www.bildindex.de/bilder/d/cv13290],[https://www.bildindex.de/media/obj06313290/cv...,[],[]


### File name to link

In [ ]:
pic = 'img-fmd463147'
imgd.filename_to_link(pic)

'http://www.bildindex.de/bilder/d/fmd463147'

### File name to all objects, to which it belongs

In [211]:
file_name = 'img-fmd463147'
df.iloc[imgd.filename_to_object(df_mvp, file_name)]

,id_record,id_persistent,label,image_links,archive_links,microfiche_links,microfiche_archive_links
8447,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002061779,Landgrafenschloss Landgrafenschloss/Leutehaus,"[http://www.bildindex.de/bilder/d/fmd10037402,...",[https://www.bildindex.de/media/obj20269517/fm...,"[http://www.bildindex.de/bilder/d/mi12611e11, ...",[https://www.bildindex.de/media/obj20269517/mi...
8448,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002061837,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463125, h...",[https://www.bildindex.de/media/obj20269517/fm...,[http://www.bildindex.de/bilder/d/mi02046c06],[https://www.bildindex.de/media/obj20269517/mi...
8451,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0001602483,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463141, h...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]
8452,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0001602482,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463144, h...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]
8454,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002062432,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463149, h...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]
8455,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002062433,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463144, h...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]
8456,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002075429,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd10037624,...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]
8459,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002075432,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463141, h...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]
8460,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002075433,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463144, h...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]
8462,http://www.bildindex.de/document/obj20269517,http://id.bildindex.de/thing/0002075435,Landgrafenschloss Landgrafenschloss/Leutehaus ...,"[http://www.bildindex.de/bilder/d/fmd463149, h...",[https://www.bildindex.de/media/obj20269517/fm...,[],[]


### Object id to file names

In [ ]:
i = 34710
pic = 'img-fmd463147'

file_names = imgd.object_to_filenames(df, i)

if pic[:3] == 'img':
    is_in_obj = pic in file_names['Images']
else:
    is_in_obj = pic in file_names['Scans']

print(f'Is {pic} relates to this object ({i})? {is_in_obj}')

file_names

Is img-fmd463147 relates to this object (34710)? True


{'Images': ['img-fmd10037715',
  'img-fmd10037717',
  'img-fmd10037716',
  'img-fmd10037718',
  'img-fmd10037719',
  'img-fmd10037721',
  'img-fmd10037752',
  'img-fmd10037753',
  'img-fmd10037760',
  'img-fmd10037111',
  'img-fmd10037110',
  'img-fmd10037112',
  'img-fmd10037113',
  'img-fmd10036864',
  'img-fmd10037744',
  'img-fmd10037745',
  'img-fmd10037743',
  'img-fmd10037746',
  'img-fmc202883',
  'img-fmc427503',
  'img-fmc427501',
  'img-fmc428746',
  'img-fmd10037052',
  'img-fmd10037053',
  'img-fmb25812_14',
  'img-fmz35538',
  'img-fmd10037054',
  'img-fmd10037748',
  'img-fmd10037749',
  'img-fmd10037750',
  'img-fm810103b',
  'img-fm226081',
  'img-fm810103d',
  'img-fm226082',
  'img-fm810103a',
  'img-fm810103',
  'img-fmd10037779',
  'img-fm1541481',
  'img-fm1541481',
  'img-fmd10037747',
  'img-fm810103c',
  'img-fm497874',
  'img-fm497875',
  'img-fmd10037189',
  'img-fmd10037055',
  'img-fmd10037056',
  'img-fmd10036865',
  'img-fmd10037767',
  'img-fmd471088',
 